## Part 1

### Intcode computer

In [6]:
# states are
# 0 - running
# 1 - waiting for input
# 99 - halt

mutable struct Prog
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    relative_base::Int
    state::Int
end

str2prog(s) = parse.(Int, split(s, ","))

Prog(s::String) = Prog(vcat(str2prog(s), zeros(Int, 10000)), 1, [], [], 0, 0)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        9 => 1,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(prog::Prog) = Instruction(prog.code[prog.cur])

function take(prog::Prog, mode, offset)
    if mode == 0
        return prog.code[prog.code[prog.cur + offset] + 1]
    elseif mode == 1
        return prog.code[prog.cur + offset]
    else
        return prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1]
    end
end

function update!(prog::Prog, value, offset, mode = 0)
    if mode == 0
        prog.code[prog.code[prog.cur + offset] + 1] = value
    elseif mode == 2
        prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1] = value
    end
end

function apply(prog::Prog, instruction::Instruction)
    if instruction.op == 99
        prog.state = 99
    elseif instruction.op == 1
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 + a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 2
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 * a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 3
        if isempty(prog.input)
            prog.state = 1
        else
            update!(prog, popfirst!(prog.input), 1, instruction.modes[1])
            prog.cur += 2
        end
    elseif instruction.op == 4
        push!(prog.output, take(prog, instruction.modes[1], 1))
        prog.cur += 2
    elseif instruction.op == 5
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 != 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 6
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 == 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 7
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 < a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 8
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 == a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 9
        prog.relative_base += take(prog, instruction.modes[1], 1)
        prog.cur += 2
    end
end

function run(prog::Prog, input::Vector{Int})
    prog.input = input
    prog.state = 0
    run(prog)
end

run(prog::Prog, input::Int) = run(prog, [input])

function run(prog)
    clear!(prog)
    while prog.state == 0
        instruction = Instruction(prog)
        apply(prog, instruction)
    end
    
    prog.output
end

function feed(prog::Prog, input::Vector{Int})
    prog.input = vcat(prog.input, input)
    prog.state = 0
end

clear!(prog) = (prog.output = [])

clear! (generic function with 1 method)

In [78]:
struct Maze
    m::Dict{Tuple{Int, Int}, Set{Tuple{Int, Int}}}
    walls::Set{Tuple{Int, Int}}
end

add_wall(m::Maze, wall) = push!(m.walls, wall)
function add_coridor(m::Maze, coridor)
    dirs = [(-1, 0), (0, 1), (0, -1), (1, 0)]
    m.m[coridor] = Set([])
    for d in dirs
        t = coridor .+ d
        if t in keys(m.m)
            push!(m.m[coridor], t)
            push!(m.m[t], coridor)
        end
    end
end

function draw(m::Maze, droid)
    obj = vcat(collect(keys(m.m)), collect(m.walls), droid)
    
    tlx = minimum([x[1] for x in obj])
    tly = minimum([x[2] for x in obj])
    brx = maximum([x[1] for x in obj])
    bry = maximum([x[2] for x in obj])
    
    width = brx - tlx + 1
    height = bry - tly + 1
    arr = fill(' ', height, width)
    for x in keys(m.m)
        arr[x[2] - tly + 1, x[1] - tlx + 1] = '.'
    end
    for x in m.walls
        arr[x[2] - tly + 1, x[1] - tlx + 1] = '#'
    end
    arr[droid[2] - tly + 1, droid[1] - tlx + 1] = 'D'
    
    for i in height:-1:1
        println(join(arr[i, :]))
    end
end

visited(m::Maze, loc) = loc in keys(m.m) || loc in m.walls

neighbours(m::Maze, loc) = m.m[loc]
h(l1::Tuple{Int, Int}, l2::Tuple{Int, Int}) = sum(abs.(l1 .- l2))

function least(openset, scores)
    m = -1
    res = nothing
    for node in openset
        if m < 0 || scores[node][2] < m
            m = scores[node][2]
            res = node
        end
    end
    
    res
end

function move2command(x, y)
    m = Dict(
        (0, 1) => 1,
        (0, -1) => 2,
        (-1, 0) => 3,
        (1, 0) => 4
    )
    
    m[y .- x]
end

function reconstruct_path(came_from, current)
    path = Int[]
    prev = current
    while current in keys(came_from)
        current = came_from[current]
        push!(path, move2command(current, prev))
        prev = current
    end
    
    reverse(path)
end

function astar(m::Maze, start, goal)
    openset = Set([start])
    scores = Dict(start => [0, h(start, goal)])
    came_from = Dict{Tuple{Int, Int}, Tuple{Int, Int}}()
    while !isempty(openset)
        current = least(openset, scores)
        if current == goal
            return reconstruct_path(came_from, goal)
        end
        
        delete!(openset, current)
        for neighbour in neighbours(m, current)
            tentative_g_score = scores[current][1] + 1
            neighbour_g_score = get!(scores, neighbour, [-1, h(neighbour, goal)])[1]
            if neighbour_g_score < 0 || neighbour_g_score > tentative_g_score
                came_from[neighbour] = current
                scores[neighbour][1] = tentative_g_score
                if !(neighbour in openset) push!(openset, neighbour) end
            end
        end
    end
end

astar (generic function with 1 method)

In [185]:
prog = Prog(readline("input.txt"))
run(prog)

maze = Maze(Dict(), Set())
goto = Vector{Tuple{Tuple{Int, Int}, Tuple{Int, Int}}}()
current = (0, 0)
add_coridor(maze, current)
dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
for d in dirs
    push!(goto, (current, current .+ d))
end 

oxygen = (0, 0)

cnt = 0
while !isempty(goto)
    check = pop!(goto)
    if visited(maze, check[2]) continue end
    cnt += 1
    
    path = astar(maze, current, check[1])
    push!(path, move2command(check[1], check[2]))
    res = run(prog, path)[end]
    if res == 1
        current = check[2]
        add_coridor(maze, current)
        for d in dirs
            t = current .+ d
            if !visited(maze, t)
                push!(goto, (current, t))
            end
        end
    elseif res == 0
        current = check[1]
        add_wall(maze, check[2])
    else
        current = check[2]
        oxygen = check[2]
        add_coridor(maze, current)
        for d in dirs
            t = current .+ d
            if !visited(maze, t)
                push!(goto, (current, t))
            end
        end        
    end
    #if cnt == 226 break end
end

In [186]:
draw(maze, current)

 ##### ##### ### ############# ######### 
#.....#.....#...#.............#.........#
 ##.#.###.#.#.#.#.#####.#####.###.#####.#
#...#...#.#...#.#.#...#.#.....#...#.....#
#.#####.#.#.#####.#.#.#.#.#####.###.#### 
#.#...#...#.#...#...#.#.#.....#...#.#...#
#.#.#.#######.#.#####.#.#####.###.#.###.#
#...#.........#.......#.....#.....#...#.#
#.## ######################.###### ##.#.#
#...#.....#.....#.#.......#...#...#...#.#
 ##.#.###.#.###.#.#.#####.###.#.#.#.###.#
#...#...#...#.#...#.#...#...#.#.#...#...#
#.#####.#####.###.#.###.###.#.#.#######.#
#.......#...#.....#...#.......#...#.....#
 ########.#.#.#### ##.###.#######.###.## 
#...#.....#.#.#...#.#...#.#.....#...#...#
#.###.#####.#.#.#.#.###.###.###.###.###.#
#.....#.#...#...#.....#.....#.#.#.......#
#.#####.#.###########.#######.#.#######.#
#.#.....#.........#.......#...#.....#...#
#.#.###.#####.#.###.#####.###.#####.#### 
#...#...#...#.#.#...#...#.........#.#...#
 ####.###.#.#.###.#####D###.#######.#.#.#
#.....#...#.#.........#.#...#.....

In [188]:
length(astar(maze, oxygen, (0, 0)))

220

In [176]:
current, goto[end]

((-8, 4), ((-7, 4), (-7, 5)))

In [177]:
path = astar(maze, current, goto[end][1])
push!(path, move2command(goto[end][1], goto[end][2]))

2-element Array{Int64,1}:
 4
 1

In [178]:
run(prog, path)

2-element Array{Int64,1}:
 1
 0

In [139]:
for d in dirs
    t = current .+ d
    println(visited(maze, t))
end

false
true
true
true


In [36]:
prog = Prog(readline("input.txt"))

Prog([3, 1033, 1008, 1033, 1, 1032, 1005, 1032, 31, 1008  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1, Int64[], Int64[], 0, 0)

In [37]:
run(prog, )

0-element Array{Int64,1}

In [40]:
run(prog, 1)

1-element Array{Int64,1}:
 0

In [49]:
mz = Maze(Dict(
    (0, 0) => Set([(1, 0)]),
    (1, 0) => Set([(0, 0), (1, 1), (2, 0)]),
    (1, 1) => Set([(1, 0), (2, 1)]),
    (2, 0) => Set([(1, 0), (2, 1)]),
    (2, 1) => Set([(1, 1), (2, 0), (3, 1)]),
    (3, 1) => Set([(2, 1)])
), Set())

Maze(Dict((0, 0) => Set([(1, 0)]),(1, 0) => Set([(0, 0), (2, 0), (1, 1)]),(3, 1) => Set([(2, 1)]),(2, 0) => Set([(1, 0), (2, 1)]),(1, 1) => Set([(1, 0), (2, 1)]),(2, 1) => Set([(3, 1), (2, 0), (1, 1)])), Set(Tuple{Int64,Int64}[]))

In [77]:
draw(mz, (0, 0))

2:4
 ...
D.. 


In [69]:
vcat(collect(keys(mz.m)), collect(mz.walls), [(0, 0)])

7-element Array{Tuple{Int64,Int64},1}:
 (0, 0)
 (1, 0)
 (3, 1)
 (2, 0)
 (1, 1)
 (2, 1)
 (0, 0)

In [66]:
vcat([(0, 0)], [(1, 1)])

2-element Array{Tuple{Int64,Int64},1}:
 (0, 0)
 (1, 1)

In [3]:
Maze(Dict(), Set())

Maze(Dict{Tuple{Int64,Int64},Set{Tuple{Int64,Int64}}}(), Set(Tuple{Int64,Int64}[]))

In [41]:
move2command((1, 0), (1, 1))

1

In [45]:
vcat([1, 2], collect(Set([3])))

3-element Array{Int64,1}:
 1
 2
 3

In [47]:
fill(' ', 3, 3)

3×3 Array{Char,2}:
 ' '  ' '  ' '
 ' '  ' '  ' '
 ' '  ' '  ' '

## Part 2

In [191]:
cells = Dict{Tuple{Int, Int}, Int}()
for x in keys(maze.m)
    cells[x] = -1
end

cells[oxygen] = 0
oxyfill = Vector{Tuple{Int, Int}}()
push!(oxyfill, oxygen)
while !isempty(oxyfill)
    cell = pop!(oxyfill)
    for neighbour in neighbours(maze, cell)
        if cells[neighbour] == -1 || cells[neighbour] > cells[cell] + 1
            cells[neighbour] = cells[cell] + 1
            push!(oxyfill, neighbour)
        end
    end
end

In [194]:
maximum(values(cells))

334